In [ ]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it
    
from functools import partial
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne

import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine
from ieeg.viz.parula import parula_map

# TODO: hmm fix these utils imports, import the funcs individually. 6/1/25.

import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm
from scipy.stats import ttest_ind

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import copy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup

from src.analysis.config import experiment_conditions

from src.analysis.utils.labeled_array_utils import (
    put_data_in_labeled_array_per_roi_subject,
    remove_nans_from_labeled_array,
    remove_nans_from_all_roi_labeled_arrays,
    concatenate_conditions_by_string,
    get_data_in_time_range
)

from src.analysis.decoding.decoding import (
    concatenate_and_balance_data_for_decoding, 
    get_and_plot_confusion_matrix_for_rois_jim,
    Decoder, 
    windower,
    get_confusion_matrices_for_rois_time_window_decoding_jim,
    compute_accuracies,
    perform_time_perm_cluster_test_for_accuracies,
    plot_accuracies,
    mixup2
)

from src.analysis.spec.wavelet_functions import get_uncorrected_wavelets, get_uncorrected_multitaper, get_sig_tfr_differences, plot_mask_pages
from src.analysis.spec.subjects_tfr_objects_functions import load_or_make_subjects_tfr_objects, get_sig_tfr_differences_per_subject, get_sig_tfr_differences_per_roi

from src.analysis.utils.general_utils import (
    make_or_load_subjects_electrodes_to_ROIs_dict,
    get_good_data,
    get_sig_chans_per_subject,
    make_sig_electrodes_per_subject_and_roi_dict,
    calculate_total_electrodes,
    check_sampling_rates
)

import mne.time_frequency
from ieeg.calc.scaling import rescale
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np
from sklearn.model_selection import StratifiedKFold
from typing import Union, List, Sequence, Dict
import doctest

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)



['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\

#### 0. Load data.   
Need a way to load in the frequency information too, not just trials x channels x timepoints. Because I'm going to use frequency as a decoding feature too. For each electrode, for each training set, I think I can just mask the multitaper with the significant clusters and use that as the decoding feature, and then concatenate across electrodes to build the full training matrix.

In [2]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']

# params - these will become input variables once i functionalize this stuff
subjects = ['D0103']
signal_times = [-1.0, 1.5]
acc_trials_only = False
error_trials_only = False
stat_func = partial(ttest_ind, equal_var=False, nan_policy='omit')
p_thresh = 0.2
ignore_adjacency = 1 # ignore the channels dimension for clusters, just find clusters over frequency and time
n_perm = 10
n_jobs = 1
freqs = np.arange(2, 200., 4.)
n_cycles = freqs / 2
return_itc = False
time_bandwidth=10 
spec_method = 'multitaper'
average=False
seed=None
tails=2
n_splits=2
n_repeats=1
random_state=42

# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = r'C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')
HOME = os.path.expanduser("~")

USER = os.path.basename(HOME)

if os.name == 'nt':  # Windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
elif sys.platform == 'darwin':  # macOS
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
else:  # Linux (cluster)
    # Check if we're on the cluster by looking for /cwork directory
    if os.path.exists(f"/cwork/{USER}"):
        LAB_root = f"/cwork/{USER}"
    else:
        # Fallback for other Linux systems
        LAB_root = os.path.join(HOME, "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)

task='GlobalLocal'
conditions = experiment_conditions.stimulus_big_letter_conditions # set this to whichever conditions you're running

stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = list(conditions.keys()) # get the condition names as a list

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from C:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [3]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [4]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0103


get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [5]:
# rois_dict = {
#     'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# rois_dict = {
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
# }

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
}

rois = list(rois_dict.keys())
all_electrodes_per_subject_roi, sig_electrodes_per_subject_roi = make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LAS

In [6]:
print('total sig elecs:', sum(len(sig_chans_per_subject[sub]) for sub in sig_chans_per_subject))

total sig elecs: 82


get number of sig and all electrodes per subject and across subjects

In [7]:
for roi in rois:
    for sub in subjects:
        sig_elecs = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        all_elecs = all_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        print(f"Subject {sub}, ROI {roi}, Num of Sig Electrodes: {len(sig_elecs)}, Num of All Electrodes: {len(all_elecs)}")

Subject D0103, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 11
Subject D0103, ROI occ, Num of Sig Electrodes: 3, Num of All Electrodes: 4


In [8]:
# Example usage:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, all_electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 8
Total number of lpfc electrodes across all subjects: 265
Total number of significant occ electrodes across all subjects: 3
Total number of occ electrodes across all subjects: 73


#### 1. For each electrode, make multitaper using all training trials, for both conditions to be compared (this can only do two conditions, can't do more rn)

TODO: Need to loop over the trials and use some as training set, and have some held out as a test set that isn't used to find clusters. Maybe do this in the stats step though.

In [9]:
subjects_tfr_objects = load_or_make_subjects_tfr_objects(
    layout=layout,
    spec_method=spec_method,
    conditions_save_name=conditions_save_name,
    subjects=subjects,
    conditions=conditions,
    signal_times=signal_times,
    freqs=freqs,
    n_cycles=n_cycles,
    time_bandwidth=time_bandwidth,
    return_itc=return_itc,
    n_jobs=n_jobs,
    average=average,
    acc_trials_only=acc_trials_only,
    error_trials_only=error_trials_only 
)

Found existing TFR data. Loading from: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\spec\multitaper\subjects_tfr_objects\stimulus_big_letter_conditions_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_1_subjects_multitaper


#### 2. Find clusters that are significantly different between the two conditions, in the multitaper spectrogram  
do this for all channels per subject, and then also for all channels in an roi across subjects


In [10]:
# # THIS TAKES FOREVER DON'T RUN IT, JUST FOR DEBUGGING SO I CAN PLOT THE SIG CLUSTERS

# # For per-subject analysis (no electrode filtering needed)
# sig_elec_masks_per_subject, sig_elec_pvals_per_subject = get_sig_tfr_differences_per_subject(subjects_tfr_objects=subjects_tfr_objects, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# all_elec_masks_per_subject, all_elec_pvals_per_subject = get_sig_tfr_differences_per_subject(subjects_tfr_objects=subjects_tfr_objects, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# # For per-ROI analysis (with electrode filtering)
# sig_elec_masks_per_roi, sig_elec_pvals_per_roi = get_sig_tfr_differences_per_roi(subjects_tfr_objects=subjects_tfr_objects, electrodes_per_subject_roi=sig_electrodes_per_subject_roi, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

# all_elec_masks_per_roi, all_elec_pvals_per_roi = get_sig_tfr_differences_per_roi(subjects_tfr_objects=subjects_tfr_objects, electrodes_per_subject_roi=all_electrodes_per_subject_roi, condition_names=condition_names, stat_func=stat_func, p_thresh=p_thresh, n_perm=n_perm, ignore_adjacency=ignore_adjacency, n_jobs=n_jobs, seed=seed, tails=tails)

plot these clusters

In [11]:
# # TODO: Update the mean differences to be per subject and per roi, currently copied over from wavelet_differences.
# first_sub = subjects[0]
# first_condition = list(subjects_tfr_objects[first_sub].keys())[0]
# ch_names = subjects_tfr_objects[first_sub][first_condition].ch_names
# times = subjects_tfr_objects[first_sub][first_condition].times
# freqs = subjects_tfr_objects[first_sub][first_condition].freqs

# subjects_tfr_objects_save_dir = os.path.join(layout.root, 'derivatives', 'spec', spec_method, 'subjects_tfr_objects')

# # Now plot the mask pages:
# for sub in subjects:
#     sig_elecs_mask = sig_elec_masks_per_subject[sub]
#     sig_elecs_mask_pages = plot_mask_pages(sig_elecs_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(sig_elecs_mask_pages):
#         fig_name = f"{sub}_sig_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

#     all_elecs_mask = all_elec_masks_per_subject[sub]
#     all_elecs_mask_pages = plot_mask_pages(all_elecs_mask,
#                         ch_names,
#                         times=times,
#                         freqs=freqs,
#                         channels_per_page=60,
#                         grid_shape=(6, 10),
#                         cmap=parula_map,
#                         title_prefix=f"{sub} ",
#                         log_freq=True,
#                         show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(all_elecs_mask_pages):
#         fig_name = f"{sub}_all_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

# for roi in rois:
#     sig_elecs_roi_mask = sig_elec_masks_per_roi[roi]
#     sig_elecs_roi_mask_pages = plot_mask_pages(sig_elecs_roi_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} {roi} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(sig_elecs_roi_mask_pages):
#         fig_name = f"{sub}_{roi}_sig_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)

#     all_elecs_roi_mask = all_elec_masks_per_roi[roi]
#     all_elecs_roi_mask_pages = plot_mask_pages(all_elecs_roi_mask,
#                     ch_names,
#                     times=times,
#                     freqs=freqs,
#                     channels_per_page=60,
#                     grid_shape=(6, 10),
#                     cmap=parula_map,
#                     title_prefix=f"{sub} {roi} ",
#                     log_freq=True,
#                     show=False)

#     # Save each page as a separate figure file:
#     for i, fig in enumerate(all_elecs_roi_mask_pages):
#         fig_name = f"{sub}_{roi}_all_elecs_sig_{spec_method}_clusters_{conditions_save_name}_page_{i+1}.png"
#         fig_pathname = os.path.join(subjects_tfr_objects_save_dir, fig_name)
#         fig.savefig(fig_pathname, bbox_inches='tight')
#         print("Saved figure:", fig_name)
            
# # # get the mean differences themselves and plot them
# # mean_diff_inc_vs_con = mean_diff(incongruent_spec._data, congruent_spec._data, axis=0)
# # mean_diff_switch_vs_repeat = mean_diff(switch_spec._data, repeat_spec._data, axis=0)

# # # Now, plot the mean differences directly:
# # congruency_mean_diff_pages = plot_mask_pages(
# #     mean_diff_inc_vs_con,
# #     incongruent_spec.ch_names,
# #     times=incongruent_spec.times,
# #     freqs=incongruent_spec.freqs,
# #     grid_shape=(6, 10),
# #     cmap=parula_map,  # play with color maps
# #     title_prefix=f"{sub} Mean Inc-Con Diff: ",
# #     log_freq=True,
# #     show=False
# # )

# # # Save each page as a separate figure file:
# # for i, fig in enumerate(congruency_mean_diff_pages):
# #     if rescaled:
# #         fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_rescaled_page_{i+1}.jpg"
# #     else:
# #         fig_name = f"{sub}_inc-con_mean_diff_multitaper_{conditions_and_output_names_and_events['incongruent']['output_name']}-{conditions_and_output_names_and_events['congruent']['output_name']}_uncorrected_page_{i+1}.jpg"
# #     fig_pathname = os.path.join(save_dir, fig_name)
# #     fig.savefig(fig_pathname, bbox_inches='tight')
# #     print("Saved figure:", fig_name)

let's try creating roi labeled arrays and then masking these and decoding using them

make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **freqs**: This is the number of frequencies in the roi, each frequency is labeled as frequency name.
          - **samples**: 1 sample as a float. This is the time for this sample.


In [22]:
# Example usage:

# choose whether to use sig elecs or all elecs
electrodes = all_electrodes_per_subject_roi # toggle this to sig_electrodes_per_subject_roi if just using sig elecs, or electrodes_per_subject_roi if using all elecs

if electrodes == all_electrodes_per_subject_roi:
    elec_string_to_add_to_filename = 'all_elecs'
elif electrodes == sig_electrodes_per_subject_roi:
    elec_string_to_add_to_filename = 'sig_elecs'
else:
    elec_string_to_add_to_filename = None

roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(
    subjects_tfr_objects,
    condition_names,
    rois,
    subjects,
    electrodes, 
    obs_axs=0,  # Trials dimension (ignoring the conditions dimension for now cuz we iterate over it)
    chans_axs=1,  # Channels dimension
    freq_axs=2,
    time_axs=3,   # Time dimension
    random_state=42  # For reproducibility
)

ROI 'lpfc': Maximum trials per condition:
  Condition 'Stimulus_bigS': 176 trials from subjects ['D0103']
  Condition 'Stimulus_bigH': 183 trials from subjects ['D0103']
in roi lpfc, subject D0103 has 176 trials for condition Stimulus_bigS
in roi lpfc, subject D0103 has 183 trials for condition Stimulus_bigH
Subject D0103, ROI lpfc, LabeledArray shape: (2, 183, 11, 3, 142)
ROI 'occ': Maximum trials per condition:
  Condition 'Stimulus_bigS': 176 trials from subjects ['D0103']
  Condition 'Stimulus_bigH': 183 trials from subjects ['D0103']
in roi occ, subject D0103 has 176 trials for condition Stimulus_bigS
in roi occ, subject D0103 has 183 trials for condition Stimulus_bigH
Subject D0103, ROI occ, LabeledArray shape: (2, 183, 4, 3, 142)


In [ ]:
# move all this into decoding.py once i get it working

def decode_on_sig_tfr_clusters(
    X_train_raw, y_train, X_test_raw,
    train_indices, 
    roi_labeled_array, condition_names, obs_axs, chans_axs,
    stat_func, p_thresh, n_perm,
    Decoder, cats, explained_variance, oversample,
    ignore_adjacency=1, seed=42, tails=2, alpha=0.2
):
    """
    Balance data and decode with TFR cluster masking.
    
    Parameters:
    -----------
    X_train_raw : np.ndarray
        Raw training data (trials, channels, freqs, times)
    y_train : np.ndarray
        Training labels
    X_test_raw : np.ndarray
        Raw test data
    train_indices : np.ndarray
        Indices of training trials in the original roi_labeled_array
    roi_labeled_array : LabeledArray
        The full roi labeled array for extracting channel info
    condition_names : list
        List of condition names to compare
    obs_axs : int
        Axis of the roi_labeled_array that contains trial labels
    chans_axs : int
        Axis of the roi_labeled_array that contains channel labels
    stat_func : function
        Statistical function for cluster computation
    p_thresh : float
        P-value threshold for significance
    n_perm : int
        Number of permutations
    Decoder : class
        Decoder class to use for decoding
    cats : list
        List of category names
    explained_variance : float
        Proportion of variance to explain with PCA
    oversample : bool
        Whether to oversample the training data
    ignore_adjacency : int
        Whether to ignore adjacency in clustering
    seed : int
        Random seed
    tails : int
        Number of tails for statistical test
    alpha : float
        Alpha parameter for mixup augmentation
    """
    
    
    # Get channel labels from the labeled array as a list
    channel_labels = roi_labeled_array.labels[chans_axs+1] # add 1 to the chans axs because of the conditions axis that got addd
    
    # Step 1: Create training-only TFR masks
    channel_masks = compute_sig_tfr_masks_from_roi_labeled_array(
        roi_labeled_array, train_indices, condition_names, 
        obs_axs, chans_axs,
        stat_func, p_thresh, n_perm, 
        ignore_adjacency, seed, tails
    )
    
    # Step 2: Apply masks and flatten (now passing channel_labels)
    X_train_masked = apply_tfr_masks_and_flatten_to_make_decoding_matrix(X_train_raw, obs_axs, chans_axs, channel_masks, channel_labels)
    X_test_masked = apply_tfr_masks_and_flatten_to_make_decoding_matrix(X_test_raw, obs_axs, chans_axs, channel_masks, channel_labels)
    
    # Step 3: Decode
    decoder = Decoder(cats, explained_variance=explained_variance, n_splits=1, n_repeats=1, oversample=oversample)
    
    # Handle NaN filling using existing mixup2 function
    mixup2(arr=X_train_masked, labels=y_train, obs_axs=obs_axs, alpha=alpha, seed=seed)
    
    # Fill test NaNs with noise (as done in sample_fold)
    is_nan = np.isnan(X_test_masked)
    X_test_masked[is_nan] = np.random.normal(0,1,np.sum(is_nan))

    # Fit and predict
    decoder.fit(X_train_masked, y_train)
    preds = decoder.predict(X_test_masked)
    
    return preds

def compute_sig_tfr_masks_from_roi_labeled_array(
    roi_labeled_array, train_indices, condition_names,
    obs_axs, chans_axs, stat_func, p_thresh, n_perm, 
    ignore_adjacency=1, seed=42, tails=2
):
    """
    Compute significant TFR clusters using only training trials from roi_labeled_array.
    
    Returns:
    --------
    channel_masks : dict
        {channel_label: mask_array} where mask is (n_freqs, n_times)
    """
    channel_masks = {}
    
    # Get channel labels from the labeled array
    channel_labels = roi_labeled_array.labels[chans_axs+1]

    # Validate we have exactly 2 conditions for now
    if len(condition_names) != 2:
        raise ValueError(
            f"For now, just doing perm test instead of ANOVA, "
            f"so this will only work for two conditions. Got {len(condition_names)} conditions."
        )

    # Split training data by condition
    train_data_by_condition = {}
    for cond in condition_names:  # hmm the stats only work for two conditions, so just do two conditions for now. Can expand to >2 conditions in the future, would just need to do ANOVA i think instead of time perm cluster.
        # Extract training trials for this condition
        cond_data = roi_labeled_array[cond]  # Shape: (trials, channels, freqs, times). Test this!
        cond_train_data = np.take(cond_data, train_indices, axis=obs_axs) # TODO: keep going through this code 4:45 on 8/1 - huh? this is a 4d array, check what train_indices is. Should grab along the trials axis. Maybe do np.take(train_indices, axis=obs_axs) to be safe.
        train_data_by_condition[cond] = cond_train_data
    
    # For each channel, compute significant clusters
    n_channels = len(channel_labels)
    for ch_idx in range(n_channels):
        ch_label = channel_labels[ch_idx]
        
        # Get data for this channel across conditions
        cond_data_this_chan = {}
        for cond in condition_names:
            cond_data_this_chan[cond] = np.take(train_data_by_condition[cond], ch_idx, axis=chans_axs)
        
        # TODO: Once I implement ANOVA, don't hard code cond1 and cond2 data, just use the cond_data_this_chan dict and feed it into some get_sig_tfr_differences function that implements ANOVA instead of time perm cluster. 
        cond0_data = cond_data_this_chan[condition_names[0]]
        cond1_data = cond_data_this_chan[condition_names[1]]

        # Run time perm cluster (TODO: implement ANOVA too as another option for when i'm decoding more than two conditions)
        if len(cond0_data) > 0 and len(cond1_data) > 0:
            mask, _ = get_sig_tfr_differences(
                cond0_data, cond1_data,
                stat_func=stat_func,
                p_thresh=p_thresh,
                n_perm=n_perm,
                axis=obs_axs,  # trials axis
                ignore_adjacency=ignore_adjacency,
                seed=seed,
                tails=tails
            )
            channel_masks[ch_label] = mask
        else:
            # No data for comparison
            channel_masks[ch_label] = np.zeros(
                (cond1_data.shape[1], cond1_data.shape[2]), dtype=bool
            )
            raise ValueError(f"cond 0 has {len(cond0_data)} trials and cond 1 has {len(cond1_data)} trials. Can't decode! ")
    
    return channel_masks

def apply_tfr_masks_and_flatten_to_make_decoding_matrix(data, obs_axs, chans_axs, channel_masks, channel_labels):
    """
    Apply channel-specific TFR masks and flatten feature matrices into vectors for each trial, ultimately resulting in a trials x features matrix
    
    Parameters:
    -----------
    data : np.ndarray
        Shape: (n_trials, n_channels, n_freqs, n_times)
    obs_axs : int
        Axis of the data that contains trial labels
    chans_axs : int
        Axis of the data that contains channel labels
    channel_masks : dict
        {channel_label: mask_array} where channel_label is like "D0103-LPFC1"
    channel_labels : list
        List of channel labels in the same order as data's channel dimension
    
    Returns:
    --------
    decoding_matrix : np.ndarray
        Shape: (n_trials, n_features)
    """
    n_trials, n_channels = data.shape[obs_axs], data.shape[chans_axs]
    feature_vectors = []
    
    # Move trials to first axis if needed - TODO: check if this is necessary..?
    if obs_axs != 0:
        data = np.moveaxis(data, obs_axs, 0)
        chans_axs = chans_axs - 1 if chans_axs > obs_axs else chans_axs

    # Iterate through each channel
    for ch_idx in range(n_channels):
        # Extract this channel's data for all trials
        # Shape changes from (n_trials, n_channels, n_freqs, n_times) 
        # to (n_trials, n_freqs, n_times)
        channel_data = np.take(data, ch_idx, chans_axs)
        
        # Get the label for this channel index
        ch_label = channel_labels[ch_idx]  # e.g., "D0103-LPFC1"
        
        # Check if we have a mask for this channel
        if ch_label in channel_masks:
            # Get the boolean mask (n_freqs, n_times)
            mask = channel_masks[ch_label]
            
            # Apply mask to each trial's data
            # channel_data[:, mask] selects only the True positions in mask, because mask is n freqs x n times
            # This reduces (n_trials, n_freqs, n_times) to (n_trials, n_significant_features)
            masked_features = channel_data[:, mask]
            
            # Note: reshape(n_trials, -1) is redundant here since it's already 2D
            # masked_features is already (n_trials, n_features)
        else:
            # No mask for this channel - use all time-frequency points
            # Flatten from (n_trials, n_freqs, n_times) to (n_trials, n_freqs*n_times)
            masked_features = channel_data.reshape(n_trials, -1)
        
        # Add this channel's features to our list
        feature_vectors.append(masked_features)
    
    # Concatenate all channels' features horizontally
    # If channel 1 has 50 features and channel 2 has 30 features,
    # result will have 80 features total
    decoding_matrix = np.concatenate(feature_vectors, axis=chans_axs)
    return decoding_matrix

def get_confusion_matrix_for_rois_tfr_cluster(
    roi_labeled_arrays, rois, strings_to_find, stat_func, 
    Decoder, explained_variance=0.95,
    p_thresh=0.05, n_perm=100, 
    n_splits=5, n_repeats=5, obs_axs=0, chans_axs=1,
    balance_method='subsample', oversample=False,
    random_state=42, alpha=0.2
):
    """
    Modified version that uses TFR cluster masking.
    
    Parameters:
    -----------
    roi_labeled_arrays : dict
        Dictionary of labeled arrays by ROI
    rois : list
        List of ROIs to process
    strings_to_find : list
        List of condition strings to find
    stat_func : function
        Statistical function for cluster computation
    Decoder : class
        Decoder class to use. Not sure if this will work with other decoders besides PCA-LDA.
    explained_variance : float
        Variance to explain in PCA
    p_thresh : float
        P-value threshold
    n_perm : int
        Number of permutations
    n_splits : int
        Number of CV splits
    n_repeats : int
        Number of CV repeats
    obs_axs : int
        Observation axis
    chans_axs : int
        Channel axis
    balance_method : str
        Method for balancing ('subsample' or 'oversample')
    oversample : bool
        Whether to oversample in decoder
    random_state : int
        Random seed
    alpha : float
        Mixup alpha parameter
    
    Returns:
    --------
    confusion_matrices : dict
        Dictionary of confusion matrices by ROI
    """
    confusion_matrices = {}
    
    for roi in rois:
        print(f"Processing ROI: {roi}")
        
        # Get data and labels
        concatenated_data, labels, cats = concatenate_and_balance_data_for_decoding(
            roi_labeled_arrays, roi, strings_to_find, obs_axs, balance_method, random_state
        )
        
        # Set up cross-validation
        all_cms = []
        
        for repeat in range(n_repeats):
            repeat_seed = random_state + repeat * 1000
            skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=repeat_seed)
            
            fold_cms = []
            
            for fold_idx, (train_indices, test_indices) in enumerate(skf.split(concatenated_data, labels)):
                print(f"  Repeat {repeat+1}/{n_repeats}, Fold {fold_idx+1}/{n_splits}")
                
                # Get train/test data
                X_train_raw = concatenated_data[train_indices]
                X_test_raw = concatenated_data[test_indices]
                y_train = labels[train_indices]
                y_test = labels[test_indices]
                
                # Balance and decode with TFR masking
                preds = decode_on_sig_tfr_clusters(
                    X_train_raw, y_train, X_test_raw, y_test,
                    train_indices,
                    roi_labeled_arrays[roi], condition_names, obs_axs, chans_axs,
                    stat_func, p_thresh, n_perm,
                    Decoder, cats, explained_variance, oversample,
                    ignore_adjacency=1, seed=repeat_seed + fold_idx, tails=2, alpha=alpha
                )
                
                cm = confusion_matrix(y_test, preds)
                fold_cms.append(cm)
            
            # Sum across folds
            repeat_cm = np.sum(fold_cms, axis=0)
            all_cms.append(repeat_cm)
        
        # Average across repeats
        final_cm = np.mean(all_cms, axis=0)
        confusion_matrices[roi] = final_cm
    
    return confusion_matrices

In [4]:
doctest.testmod(verbose=True)

Trying:
    import mne
Expecting nothing
ok
Trying:
    import numpy as np
Expecting nothing
ok
Trying:
    from typing import Dict, List
Expecting nothing
ok
Trying:
    ch_names = ['CH1', 'CH2', 'CH3', 'CH4', 'CH5']
Expecting nothing
ok
Trying:
    sfreq = 200.0
Expecting nothing
ok
Trying:
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='seeg')
Expecting nothing
ok
Trying:
    n_epochs, n_freqs, n_times = 2, 10, 50
Expecting nothing
ok
Trying:
    data = np.random.randn(n_epochs, len(ch_names), n_freqs, n_times)
Expecting nothing
ok
Trying:
    freqs = np.arange(10., 20., 1.)
Expecting nothing
ok
Trying:
    times = np.linspace(-0.5, 1.0, n_times)
Expecting nothing
ok
Trying:
    tfr_mock = mne.time_frequency.EpochsTFR(info, data, times, freqs)
Expecting nothing
ok
Trying:
    subjects_tfr_objects_mock = {'sub_1': {'condition_1': tfr_mock, 'condition_2': tfr_mock.copy()}}
Expecting nothing
ok
Trying:
    electrodes_per_subject_roi_mock = {
        'roi_1': {'sub_

TestResults(failed=1, attempted=16)